In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os

# Étape 1 : Charger les documents
def load_documents_from_folder(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                documents.append(file.read())
    return documents

# Étape 2 : Créer des embeddings pour les documents
def create_embeddings(documents, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(documents, convert_to_tensor=True)
    return embeddings

# Étape 3 : Recherche du passage pertinent
def search_relevant_document(question, documents, embeddings, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    question_embedding = model.encode([question], convert_to_tensor=True)
    
    similarities = cosine_similarity(question_embedding.cpu().detach().numpy(), embeddings.cpu().detach().numpy())
    
    best_doc_index = similarities.argmax()
    return documents[best_doc_index]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

def answer_question_with_bert(question, context, model_name="google/flan-t5-large"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Nettoyer et tronquer le contexte pour éviter de dépasser la limite de tokens
    context = context.strip().replace("\n", " ")
    max_context_tokens = 700  # Laisse de la place pour le prompt
    context_tokens = tokenizer.encode(context, truncation=True, max_length=max_context_tokens)
    truncated_context = tokenizer.decode(context_tokens, skip_special_tokens=True)

    # Construire un prompt clair et structuré
    prompt = (
        f"Tu es un expert médical. Lis attentivement le contexte ci-dessous et réponds à la question de manière précise. "
        f"Contexte : {truncated_context}\n\n"
        f"Question : {question}\n\n"
        f"Réponse :"
    )


    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    # Génération avec paramètres de décodage contrôlés pour améliorer la qualité
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,         # Limite de tokens de la réponse
        do_sample=False,            # Pas de génération créative (pour des réponses déterministes)
        num_beams=4,                # Beam search améliore la précision
        early_stopping=True,        # Arrêt anticipé si la réponse est complète
        temperature=0.7,            # Contrôle la créativité de la réponse
        top_p=0.9                   # Top-p sampling pour plus de diversité si do_sample=True
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return answer

# Exemple d'utilisation
folder_path = 'notices_txt'
documents = load_documents_from_folder(folder_path)
embeddings = create_embeddings(documents)

question = "Qu'est-ce que CIMETIDINE ARROW 200 mg ?"
relevant_document = search_relevant_document(question, documents, embeddings)
print("Document pertinent :\n", relevant_document)

answer = answer_question_with_bert(question, relevant_document)
print("Réponse :", answer)


/home/abdellatifunix/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Document pertinent :
 Dénomination du médicament
CIMETIDINE ARROW 200 mg, comprimé effervescent
Encadré
Veuillez lire attentivement cette notice avant de prendre ce médicament car elle contient des informations importantes pour vous.
·
Gardez cette notice. Vous pourriez avoir besoin de la relire.
·
Si vous avez dautres questions, interrogez votre médecin ou votre pharmacien.
·
Ce médicament vous a été personnellement prescrit. Ne le donnez pas à dautres personnes. Il pourrait leur être nocif, même si les signes de leur maladie sont identiques aux vôtres.
·
Si vous ressentez un quelconque effet indésirable, parlez-en à votre médecin ou votre pharmacien. Ceci sapplique aussi à tout effet indésirable qui ne serait pas mentionné dans cette notice.
Voir rubrique 4.
Que contient cette notice ?
1. Qu'est-ce que CIMETIDINE ARROW 200 mg, comprimé effervescent et dans quels cas est-il utilisé ?
2. Quelles sont les informations à connaître avant de prendre CIMETIDINE ARROW 200 mg, comprimé eff

/home/abdellatifunix/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abdellatifunix/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Réponse : Classe pharmacothérapeutique : antagoniste des récepteurs H2
